## Basic Generation Training

In [ ]:
# Change to correct directory
%cd ../MedSinGAN/

In [ ]:
# Run a best combination for a given image
! python main_train.py --train_mode generation --input_name ../pipeline_data_tcia/benign/benign.png --train_stages 16 --niter 2500 --gpu 0 --train_depth 9

## Combination Generation Testing

In [ ]:
# Change to correct directory
%cd ../MedSinGAN/

In [2]:
# Required imports
import sys
sys.path.append("../utils")
import os
from utils import execute_bash_command

In [ ]:
# Parameters variation
# (Stages, N Iterations, Concurrent Stages, Activation Function)
params_variations = [
    (6,1000,3,"lrelu"),(6,1000,3,"prelu"),
    (6,1000,6,"lrelu"),(6,1000,6,"prelu"),
    (12,1000,6,"lrelu"),(12,1000,6,"prelu"),
    (12,1000,10,"lrelu"),(12,1000,10,"prelu"),
    (16,1000,9,"lrelu"),(16,1000,9,"prelu")]

In [ ]:
# Execute single experiment with given parameters
def do_single_experiment(image_pth, stages, niter, concurrent, a_func):
    cmd = f"python main_train.py --train_mode generation --input_name {image_pth} --im_max_size 614 --train_stages {stages} --niter {niter} --gpu 0 --train_depth {concurrent} --activation {a_func}"
    for path in execute_bash_command(cmd.split()):
        print(path, end="")
        
    comand = f"zip -r ../(G)S{stages}_NITER{niter}_CONC{concurrent}.zip ."
    for path in execute_bash_command(comand.split()):
        print(path, end="")
    
    ! rm -r mlruns TrainedModels

In [ ]:
# Run experimentations
for comb in params_variations:
    do_single_experiment('../pipeline_data_tcia/benign/benign.png', comb[0], comb[1], comb[2], comb[3])

## Save Model

In [ ]:
# Zip everything from this model
! zip -r current_model.zip .

## Delete Current Model

In [ ]:
# Delete everything that the model produces
! rm -r mlruns TrainedModels